In [ ]:
%pip install mesa

In [ ]:
# Importación de librerías

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

import numpy as np
import time
import datetime

## Clase para el agente, uso de vecindario y movimiento, búsqueda de celdas vacías.

class CleaningRobotAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

    def step(self):
        (x, y) = self.pos
        if self.model.dirty_cells[x][y] == 1:
            self.model.dirty_cells[x][y] = 0

        neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        vacant_cells = [cell for cell in neighborhood if self.model.grid.is_cell_empty(cell)]
        if vacant_cells:
            new_position = self.random.choice(vacant_cells)
            self.model.grid.move_agent(self, new_position)


## Clase para el modelo, uso de grid, agentes, y recolección de datos.
## Recibe: ancho, alto, número de aspiradoras, porcentaje de suciedad.

class CleaningModel(Model):
    def __init__(self, width, height, num_vacuums, dirty_percentage):
        self.num_vacuums = num_vacuums
        self.total_cells = width * height
        self.amount_of_dirty_cells = int(self.total_cells * (dirty_percentage / 100))
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.dirty_cells = np.zeros((width, height))
        self.datacollector = DataCollector(
            model_reporters={"Dirty_Percentage": lambda m: (np.sum(m.dirty_cells) / self.total_cells) * 100},
            agent_reporters={"Position": "pos"}
        )

        for i in range(self.num_vacuums):
            agent = CleaningRobotAgent(i, self)
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x, y))

        count = 0
        while count < self.amount_of_dirty_cells:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                count += 1

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


## Función para correr la simulación, uso de tiempo y pasos.
## Recibe: ancho, alto, número de aspiradoras, porcentaje de suciedad, número de pasos.


def run_simulation(width, height, num_vacuums, dirty_percentage, max_steps=None):
    print(f"Running simulation for {max_steps if max_steps else 'unlimited'} steps")
    start_time = time.time()
    model = CleaningModel(width, height, num_vacuums, dirty_percentage)
    steps = 0

    while max_steps is None or steps < max_steps:
        model.step()
        if np.sum(model.dirty_cells) == 0:
            break
        steps += 1

    dirty_percentage = (np.sum(model.dirty_cells) / model.total_cells) * 100
    print("Percentage of dirty cells:", dirty_percentage)
    print('Execution Time:', str(datetime.timedelta(seconds=(time.time() - start_time))))
    print(f'Number of steps: {steps}')


# Simulaciones para el caso 1
run_simulation(100, 100, 1, 90, 100)
run_simulation(100, 100, 2, 90, 100)
run_simulation(100, 100, 1, 90, 1000)
run_simulation(100, 100, 2, 90, 1000)
run_simulation(100, 100, 1, 90, 10000)
run_simulation(100, 100, 2, 90, 10000)

In [ ]:
#Simulaciones ´para el caso 2

run_simulation(100, 100, 1, 90)

In [ ]:
run_simulation(100, 100, 2, 90)

In [ ]:
run_simulation(100, 100, 3, 90)
run_simulation(100, 100, 4, 90)
run_simulation(100, 100, 5, 90)
run_simulation(100, 100, 6, 90)
run_simulation(100, 100, 7, 90)

In [ ]:
run_simulation(100, 100, 8, 90)

In [ ]:
run_simulation(100, 100, 9, 90)

In [ ]:
run_simulation(100, 100, 10, 90)